# Automated Essay Scoring

In [1]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices())

2.8.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# import important libraries and download data
import os
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import cohen_kappa_score, r2_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import multiprocessing
%matplotlib notebook
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')  
# ! git clone https://github.com/Gaurav-Pande/AES_DL.git && mv AES_DL/data .

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/SonTungKieu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/SonTungKieu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/SonTungKieu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# Declaring some visualization methods to plot accuracy and model diagram
def plot_loss_curve(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['mae'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

def plot_acrchitecture(filename, model):
    plot_model(model, to_file=str(filename) + '.png')

In [4]:
# method to split data into sets
def split_in_sets(data):
    essay_sets = []
    min_scores = []
    max_scores = []
    
    for s in range(1,9):
        essay_set = data[data["essay_set"] == s]
        essay_set.dropna(axis=1, inplace=True)
        n, d = essay_set.shape
        set_scores = essay_set["domain1_score"]
        print ("Set", s, ": Essays = ", n , "\t Attributes = ", d)
        min_scores.append(set_scores.min())
        max_scores.append(set_scores.max())
        essay_sets.append(essay_set)
    
    return essay_sets, min_scores, max_scores

In the below cell, we can see the data we need to operate. We essentially drops the column, we dont need and keep the domain_score only along with essay text.

In [5]:
dataset_path = "./data/training_set_rel3.tsv"
data = pd.read_csv(dataset_path, sep="\t", encoding="ISO-8859-1")
data.head()

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,NaN,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
data.shape

(12976, 28)

In [7]:
min_scores = [2, 1, 0, 0, 0, 0, 0, 0]
max_scores = [12, 6, 3, 3, 4, 4, 30, 60]
essay_sets, data_min_scores, data_max_scores = split_in_sets(data)
set1, set2, set3, set4, set5, set6, set7, set8 = tuple(essay_sets)
data.dropna(axis=1, inplace=True)
data.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set1.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set2.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set3.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set4.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set5.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set6.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set7.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set8.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
sets = [set1,set2,set3,set4,set5,set6,set7,set8]
data.head()

/var/folders/pj/ldkmdmbj5bld43hhctfjwh280000gn/T/ipykernel_48848/290102877.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  essay_set.dropna(axis=1, inplace=True)
/var/folders/pj/ldkmdmbj5bld43hhctfjwh280000gn/T/ipykernel_48848/1243294380.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  set1.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
/var/folders/pj/ldkmdmbj5bld43hhctfjwh280000gn/T/ipykernel_48848/1243294380.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

Set 1 : Essays =  1783 	 Attributes =  6
Set 2 : Essays =  1800 	 Attributes =  9
Set 3 : Essays =  1726 	 Attributes =  6
Set 4 : Essays =  1770 	 Attributes =  6
Set 5 : Essays =  1805 	 Attributes =  6
Set 6 : Essays =  1800 	 Attributes =  6
Set 7 : Essays =  1569 	 Attributes =  14
Set 8 : Essays =  723 	 Attributes =  18


/var/folders/pj/ldkmdmbj5bld43hhctfjwh280000gn/T/ipykernel_48848/1243294380.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  set3.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
/var/folders/pj/ldkmdmbj5bld43hhctfjwh280000gn/T/ipykernel_48848/1243294380.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  set4.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
/var/folders/pj/ldkmdmbj5bld43hhctfjwh280000gn/T/ipykernel_48848/1243294380.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.p

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


In [8]:
cap = ['@CAPS'+str(i) for i in range(100)]
loc = ['@LOCATION'+str(i) for i in range(100)]
org =['@ORGANIZATION'+str(i) for i in range(100)]
per = ['@PERSON'+str(i) for i in range(100)]
date = ['@DATE'+str(i) for i in range(100)]
time = ['@TIME'+str(i) for i in range(100)]
money = ['@MONEY'+str(i) for i in range(100)]
ner =  cap + loc + org + per + date + time + money

## Score distribution

In [9]:
topic_number = 0
fig, ax = plt.subplots(4,2, figsize=(7, 7), sharey=False)

for i in range(4):
    for j in range(2):
        topic_number += 1
        data[data['essay_set'] == topic_number].groupby('domain1_score')['essay_id'].agg('count').plot.bar(ax=ax[i, j], rot=0, color="maroon")
        ax[i,j].set_title('Essay %i' % topic_number)

ax[3,0].locator_params(nbins=10)
ax[3,1].locator_params(nbins=10)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

<IPython.core.display.Javascript object>

In [12]:
essay_sets, data_min_scores, data_max_scores = split_in_sets(data)
set1, set2, set3, set4, set5, set6, set7, set8 = tuple(essay_sets)
data.dropna(axis=1, inplace=True)
sets = [set1,set2,set3,set4,set5,set6,set7,set8]
set1.head()

Set 1 : Essays =  1783 	 Attributes =  4
Set 2 : Essays =  1800 	 Attributes =  4
Set 3 : Essays =  1726 	 Attributes =  4
Set 4 : Essays =  1770 	 Attributes =  4
Set 5 : Essays =  1805 	 Attributes =  4
Set 6 : Essays =  1800 	 Attributes =  4
Set 7 : Essays =  1569 	 Attributes =  4
Set 8 : Essays =  723 	 Attributes =  4


/var/folders/pj/ldkmdmbj5bld43hhctfjwh280000gn/T/ipykernel_48848/290102877.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  essay_set.dropna(axis=1, inplace=True)


,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


# Model

In [13]:
from tensorflow.keras.layers import Embedding, Input, LSTM, Dense, Dropout, Lambda, Flatten, Bidirectional, Conv2D, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.models import Sequential,Model, load_model, model_from_config
import keras.backend as K
from tensorflow.keras.optimizers import Adam

In [14]:
def get_model(Hidden_dim1=400, Hidden_dim2=128, return_sequences = True, dropout=0.3, recurrent_dropout=0.3, input_size=768, activation='relu', bidirectional = False):
    """Define the model."""
    model = Sequential()
    if bidirectional:
        model.add(Bidirectional(LSTM(Hidden_dim1,return_sequences=return_sequences , dropout=0.4, recurrent_dropout=recurrent_dropout), input_shape=[1, input_size]))
        model.add(Bidirectional(LSTM(Hidden_dim2, recurrent_dropout=recurrent_dropout, return_sequences=return_sequences)))
        model.add(Bidirectional(LSTM(Hidden_dim2, recurrent_dropout=recurrent_dropout)))
    else:
        model.add(LSTM(Hidden_dim1, dropout=0.4, recurrent_dropout=recurrent_dropout, input_shape=[1, input_size], return_sequences=return_sequences))
        model.add(LSTM(Hidden_dim2, recurrent_dropout=recurrent_dropout))

    model.add(Dropout(dropout))
    model.add(Dense(128, activation=activation))
    model.add(Dense(64, activation=activation))
    model.add(Dense(32, activation=activation))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'mae', tf.keras.losses.MeanAbsolutePercentageError()])
    model.summary()
    
    return model

In [15]:
def get_features(X):
    max_len = 0
    for i in X.values:
        if len(i) > max_len:
            max_len = len(i)

    padded_X = np.array([i + [0]*(max_len-len(i)) for i in X.values])
    attention_mask_X = np.where(padded_X != 0, 1, 0)
    X_input_ids = torch.tensor(padded_X)
    X_attention_mask = torch.tensor(attention_mask_X)

    with torch.no_grad():
        last_hidden_states_X = model(X_input_ids, attention_mask=X_attention_mask)

    X_features = last_hidden_states_X[0][:,0,:].numpy()

    return X_features


In [16]:
# Sets experiment BERT
import time
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
set_count = 1
all_sets_score = []

# Metrics
mape = tf.keras.losses.MeanAbsolutePercentageError()
mse = tf.keras.losses.MeanSquaredError()
mae = tf.keras.losses.MeanAbsoluteError()


for s in sets:
    print("\n--------SET {}--------\n".format(set_count))
    X = s
    y = s['domain1_score']
    cv = KFold(n_splits=5, shuffle=True)
    cv_data = cv.split(X)

    kappa_results = []
    mse_results = []
    mae_results = []
    mape_results = []

    prediction_list = []
    fold_count =1

#     cuda = torch.device('cuda')
    model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
#     model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

    # Load pretrained model/tokenizer
    tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
    model = model_class.from_pretrained(pretrained_weights)

    for traincv, testcv in cv_data:
        print("\n--------Fold {}--------\n".format(fold_count))

        # get the train and test from the dataset.
        X_train, X_test, y_train, y_test = X.iloc[traincv], X.iloc[testcv], y.iloc[traincv], y.iloc[testcv]
        train_essays = X_train['essay']
        test_essays = X_test['essay']

        sentences = []
        tokenize_sentences = []
        train_bert_embeddings = []

        tokenized_train = train_essays.apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=200)))
        tokenized_test = test_essays.apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=200)))

        train_features = get_features(tokenized_train)
        test_features = get_features(tokenized_test)

        train_x, train_y = train_features.shape
        test_x, test_y = test_features.shape

        trainDataVectors = np.reshape(train_features,(train_x, 1, train_y))
        testDataVectors = np.reshape(test_features,(test_x, 1, test_y))

        # Bidirectional LSTM Model
        bi_dir_lstm_model = get_model(bidirectional=True)
        history = bi_dir_lstm_model.fit(trainDataVectors, y_train, batch_size=50, epochs=100)

        # Plot loss
        plot_loss_curve(history)

        y_pred = bi_dir_lstm_model.predict(testDataVectors)
        y_pred = np.around(y_pred)
        #y_pred.dropna()
        np.nan_to_num(y_pred)

        y_test = y_test.to_numpy().reshape(-1, 1)

        y_pred = y_pred.reshape(-1, 1)

        result_tbl = pd.DataFrame(data=np.concatenate((y_test, y_pred), axis=1),
                                  columns=['Test', 'Pred'])
        print(result_tbl)

        # evaluate the model

        kappa = cohen_kappa_score(y_test, y_pred, weights='quadratic')
        mape_score = mape(y_test, y_pred).numpy()
        mse_score = mse(y_test, y_pred).numpy()
        mae_score = mae(y_test, y_pred).numpy()

        print("Kappa Score: {}".format(kappa))
        print('MSE {}'.format(mse_score))
        print('MAE {}'.format(mae_score))
        print('MAPE {}'.format(mape_score))

        kappa_results.append(kappa)
        mse_results.append(mse_score)
        mae_results.append(mae_score)
        mape_results.append(mape_score)

        fold_count +=1

        tf.keras.backend.clear_session()

        all_sets_score.append(kappa_results)

    print("Average Kappa score value is : {}".format(np.mean(np.asarray(kappa_results))))
    print("Average MSE score value is : {}".format(np.mean(np.asarray(mse_results))))
    print("Average MAE score value is : {}".format(np.mean(np.asarray(mae_results))))
    print("Average MAPE score value is : {}".format(np.mean(np.asarray(mape_results))))
    set_count+=1
        # print(features.shape)


--------SET 1--------



Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the 


--------Fold 1--------

Metal device set to: Apple M1 Pro


2022-06-01 23:42:15.117962: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-01 23:42:15.118853: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 1, 800)           3740800   
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 1, 256)           951296    
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 256)              394240    
 nal)                                                            
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                        

...

     Test  Pred
0    30.0  35.0
1    34.0  35.0
2    44.0  40.0
3    31.0  35.0
4    30.0  36.0
..    ...   ...
139  34.0  35.0
140  36.0  40.0
141  40.0  40.0
142  32.0  35.0
143  40.0  40.0

[144 rows x 2 columns]
Kappa Score: 0.3594655464736696
MSE 26.38888931274414
MAE 3.8611111640930176
MAPE 12.0646333694458

--------Fold 5--------

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 1, 800)           3740800   
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 1, 256)           951296    
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 256)              394240    
 nal)                                         

2022-06-02 03:23:45.067624: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


12/12 [==============================] - 12s 301ms/step - loss: 1255.8801 - mse: 1255.8804 - mae: 34.8731 - mean_absolute_percentage_error: 93.2041
Epoch 2/100
12/12 [==============================] - 2s 133ms/step - loss: 261.8903 - mse: 261.8903 - mae: 13.5333 - mean_absolute_percentage_error: 36.7185
Epoch 3/100
12/12 [==============================] - 1s 114ms/step - loss: 77.2643 - mse: 77.2643 - mae: 6.8208 - mean_absolute_percentage_error: 20.2877
Epoch 4/100
12/12 [==============================] - 1s 103ms/step - loss: 53.7220 - mse: 53.7220 - mae: 5.8645 - mean_absolute_percentage_error: 16.0914
Epoch 5/100
12/12 [==============================] - 1s 103ms/step - loss: 40.3457 - mse: 40.3457 - mae: 4.9275 - mean_absolute_percentage_error: 14.9478
Epoch 6/100
12/12 [==============================] - 1s 102ms/step - loss: 36.1607 - mse: 36.1607 - mae: 4.7438 - mean_absolute_percentage_error: 13.5559
Epoch 7/100
12/12 [==============================] - 1s 100ms/step - loss: 35.4

2022-06-02 03:25:47.068559: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


     Test  Pred
0    34.0  35.0
1    40.0  37.0
2    26.0  35.0
3    40.0  35.0
4    43.0  37.0
..    ...   ...
139  30.0  35.0
140  32.0  35.0
141  33.0  37.0
142  30.0  35.0
143  35.0  37.0

[144 rows x 2 columns]
Kappa Score: 0.20955718081979158
MSE 23.52083396911621
MAE 3.8958332538604736
MAPE 11.13329792022705
Average Kappa score value is : 0.4351890986816369
Average MSE score value is : 23.802635192871094
Average MAE score value is : 3.8272509574890137
Average MAPE score value is : 10.862098693847656


In [17]:
result_tbl

,Test,Pred
0,34.0,35.0
1,40.0,37.0
2,26.0,35.0
3,40.0,35.0
4,43.0,37.0
...,...,...
139,30.0,35.0
140,32.0,35.0
141,33.0,37.0
142,30.0,35.0


In [18]:
txt = 'I love my mom'

tokens = tokenizer.encode_plus(txt, add_special_tokens=True, max_length=512, truncation=True, padding="max_length")

tokens

{'input_ids': [101, 1045, 2293, 2026, 3566, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [20]:
tf.keras.backend.clear_session()

In [22]:
# x = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
def plot_qwk_scores_all_sets():
  fig = plt.figure()
  ax = plt.subplot(111)
  x = [1,2,3,4,5]
  ax.plot(x, set1 , label='set1')
  ax.plot(x, set2, label='set2')
  ax.plot(x, set3, label='set3')
  ax.plot(x, set4, label='set4')
  ax.plot(x, set5, label='set5')
  ax.plot(x, set6, label='set6')
  ax.plot(x, set7, label='set7')
  ax.plot(x, set8, label='set8')
  plt.title('Set wise QWK using BERT for individual sets')
  ax.legend()
  plt.show()
